**Table of contents**<a id='toc0_'></a>    
- [Load dữ liệu từ các thư mục train và dev](#toc1_)    
- [Pha huấn luyện](#toc2_)    
  - [Task1: Nhận diện câu so sánh](#toc2_1_)    
    - [Embbeding các câu trong dữ liệu huấn luyện](#toc2_1_1_)    
    - [Embbedding các nhãn](#toc2_1_2_)    
    - [Huấn luyện model](#toc2_1_3_)    
    - [Đánh giá model](#toc2_1_4_)    
    - [Chạy thử một vài ví dụ](#toc2_1_5_)    
  - [Task 2: Extract đầy đủ cái đặc trưng của phép so sánh.](#toc2_2_)    
    - [Tiền xử lý](#toc2_2_1_)    
    - [Huấn luyện model](#toc2_2_2_)    
- [Pha suy luận (dự đoán)](#toc3_)    
  - [Task 1: Liệu câu có so sánh hay không?](#toc3_1_)    
    - [Dự đoán nhãn](#toc3_1_1_)    
    - [Hậu xử lý](#toc3_1_2_)    
  - [Task 2: Chi tiết các vật thể được so sánh](#toc3_2_)    
    - [Dự đoán](#toc3_2_1_)    
    - [Hậu xử lý](#toc3_2_2_)    
- [Tổng hợp và lưu dữ liệu](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Load dữ liệu từ các thư mục train và dev](#toc0_)

> Dữ liệu được parse và format thành các Record (datatypes.py)

In [21]:
%load_ext autoreload
%autoreload 2

from dataloader import DataLoader

trainset = DataLoader.load_data("../data/train")
devset = DataLoader.load_data("../data/dev")

print(f"{trainset[:3]=}")
print(f"{devset[:3]=}")


trainset[:3]=[Record(original='Title: Review iPhone 14: Thiết kế, hiệu năng, camera và pin', sentence='Title : Review iPhone 14 : Thiết kế , hiệu năng , camera và pin', quintuples=[], comparative=['No']), Record(original='iPhone 14 được ra mắt vào 09/2022, được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước.', sentence='iPhone 14 được ra mắt vào 09 / 2022 , được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước .', quintuples=[{'subject': ['1&&iPhone', '2&&14'], 'object': ['28&&thế', '29&&hệ', '30&&trước'], 'aspect': ['20&&phần', '21&&cứng'], 'predicate': ['16&&nâng', '17&&cấp', '18&&nhẹ'], 'label': 'COM+'}, {'subject': ['1&&iPhone', '2&&14'], 'object': ['28&&thế', '29&&hệ', '30&&trước'], 'aspect': ['24&&phần', '25&&mềm'], 'predicate': ['16&&nâng', '17&&cấp', '18&&nhẹ'], 'label': 'COM+'}], comparative=['COM+', 'COM+']), Record(original='Về ngoại hình thì cũng có rất ít sự khác biệt so với thế hệ tiền nhiệm.', sentence='Về n

# <a id='toc2_'></a>[Pha huấn luyện](#toc0_)

## <a id='toc2_1_'></a>[Task1: Nhận diện câu so sánh](#toc0_)

Phương pháp: Sử dụng model RandomForest để nhận diện đa nhãn (do mỗi câu có thể nhiều câu so sánh)
- Embbeding các câu và label
- Huấn luyện model sử dụng dữ liệu dạng vector ở bước trên

### <a id='toc2_1_1_'></a>[Embbeding các câu trong dữ liệu huấn luyện](#toc0_)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()

sentences = [record.sentence for record in trainset]
embeded_sentences = tfidf_vec.fit_transform(sentences)

print(embeded_sentences.shape)

(4171, 2464)


### <a id='toc2_1_2_'></a>[Embbedding các nhãn](#toc0_)

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

labels = [record.comparative for record in trainset]
label_classes = ["No", "DIF", "EQL", "SUP+", "SUP-", "SUP", "COM+", "COM-", "COM"]

multilabel_model = MultiLabelBinarizer(classes=label_classes)
embeded_labels = multilabel_model.fit_transform(labels)

### <a id='toc2_1_3_'></a>[Huấn luyện model](#toc0_)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler

x_train, x_val, y_train, y_val = train_test_split(embeded_sentences, embeded_labels, test_size=0.2)


In [5]:
print(f"{x_train.shape=}")
print(f"{x_train.shape=}")
print(embeded_labels[:10])

x_train.shape=(3336, 2464)
x_train.shape=(3336, 2464)
[[1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]]


In [6]:
from sklearn.multioutput import MultiOutputClassifier

randomforest_model = RandomForestClassifier(n_estimators = 500, criterion = 'entropy',max_depth = 10, class_weight='balanced')
multioutput_classifier = MultiOutputClassifier(randomforest_model)
multioutput_classifier.fit(x_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                       criterion='entropy',
                                                       max_depth=10,
                                                       n_estimators=500))

### <a id='toc2_1_4_'></a>[Đánh giá model](#toc0_)

In [7]:
y_preds = multioutput_classifier.predict(x_val)

print('Accuracy = ', accuracy_score(y_val, y_preds))
report = classification_report(y_val, y_preds)
print(report)

Accuracy =  0.8011976047904191
              precision    recall  f1-score   support

           0       0.94      0.91      0.92       673
           1       0.00      0.00      0.00         7
           2       0.62      0.48      0.54        52
           3       0.67      0.27      0.38        15
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.60      0.63      0.61        73
           7       1.00      0.11      0.19        19
           8       0.00      0.00      0.00         3

   micro avg       0.88      0.82      0.85       842
   macro avg       0.43      0.27      0.29       842
weighted avg       0.87      0.82      0.84       842
 samples avg       0.82      0.82      0.82       842



/home/ppvan/Documents/code/academic/data-mining/datamine/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppvan/Documents/code/academic/data-mining/datamine/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ppvan/Documents/code/academic/data-mining/datamine/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` paramet

### <a id='toc2_1_5_'></a>[Chạy thử một vài ví dụ](#toc0_)

In [8]:
sample_sentence = "Dù là chip \" xịn xò \" nhất so với các điện thoại Android nhưng khi so sánh với iPhone iPhone 13 Pro Max có phần chiếm ưu thế hơn ."
embeded_sentence = tfidf_vec.transform([sample_sentence])
prediction = multioutput_classifier.predict(embeded_sentence)
print(multilabel_model.inverse_transform(prediction))
print(prediction)

[('COM+',)]
[[0 0 0 0 0 0 1 0 0]]


## <a id='toc2_2_'></a>[Task 2: Extract đầy đủ cái đặc trưng của phép so sánh.](#toc0_)

**Phương pháp** : Conditional Random Fields (CRF)

Sử dụng kĩ thuật POSTag để dự đoán thành phần trong câu, từ đó suy ra các chủ thể của phép so sánh.

=> Xác định được POS-tag của từng từ trong câu.
=> Xác định được entity(subject, object), aspect, predicate

Ví dụ: Javascript có hiệu năng kém hơn C

=> ["B-Sub", "O", "B-Asp", "I-Asp", "B-Pre", "I-Pre", "B-Sub"]

=> Như dữ liệu train, lười viết.



- Tiền xử lý dữ liệu (dạng CRF hiểu được)
- Huấn luyện model

**Vấn đề:**

Subject trùng với Object --> Hết cứu


### <a id='toc2_2_1_'></a>[Tiền xử lý](#toc0_)

In [10]:
from processing import RecordEncoder
from utils import dupplicate_record_by_quintuple

cleaned_trainset = []
for record in trainset:
    if not record.quintuples:
        continue

    cleaned_trainset.extend(dupplicate_record_by_quintuple(record))

encoder = RecordEncoder()

x = [encoder.record2features(record) for record in cleaned_trainset]
y = [encoder.record2tags(record) for record in cleaned_trainset]


### <a id='toc2_2_2_'></a>[Huấn luyện model](#toc0_)

In [11]:
from sklearn_crfsuite import CRF

crf = CRF(
    algorithm="lbfgs",
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)

crf.fit(x, y)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

# <a id='toc3_'></a>[Pha suy luận (dự đoán)](#toc0_)

## <a id='toc3_1_'></a>[Task 1: Liệu câu có so sánh hay không?](#toc0_)

### <a id='toc3_1_1_'></a>[Dự đoán nhãn](#toc0_)

In [13]:
dev_embeded_sentences = tfidf_vec.transform([record.sentence for record in devset])

print(dev_embeded_sentences.shape)

comparative_preditions = multioutput_classifier.predict(embeded_sentences)
comparative_preditions.shape


(1733, 2464)


(4171, 9)

### <a id='toc3_1_2_'></a>[Hậu xử lý](#toc0_)

Decode vector về dạng string

Trong trường hợp model không đưa ra label nào, ta mặc định nó không có câu so sánh ("No")

In [15]:
readable_comparative_preditions = multilabel_model.inverse_transform(comparative_preditions)

formatted_comparative_predictions = []

for prediction in readable_comparative_preditions:
    if prediction:
        formatted_comparative_predictions.append(list(prediction))
    else:
        formatted_comparative_predictions.append(["No"])

formatted_comparative_predictions[:5]

[['No'], ['COM+'], ['No'], ['No'], ['No']]

## <a id='toc3_2_'></a>[Task 2: Chi tiết các vật thể được so sánh](#toc0_)

### <a id='toc3_2_1_'></a>[Dự đoán](#toc0_)

In [16]:
from processing import RecordDecoder

dev_featured_sentences = [encoder.record2features(record) for record in devset]
decoder = RecordDecoder()
postag_preditions = crf.predict(dev_featured_sentences)


### <a id='toc3_2_2_'></a>[Hậu xử lý](#toc0_)

In [17]:
readable_postag_predictions = [[decoder.parse_tag(_id) for _id in postag_prediction] for postag_prediction in postag_preditions]
readable_postag_predictions[0]


['O',
 ':',
 'O',
 'O',
 'B-Sub',
 'I-Sub',
 'I-Sub',
 'I-Sub',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 '.']

# <a id='toc4_'></a>[Tổng hợp và lưu dữ liệu](#toc0_)

In [26]:
labeled_records = [
    decoder.label_record(
        record, formatted_comparative_predictions[i], readable_postag_predictions[i]
    )
    for i, record in enumerate(devset)
]

for record in labeled_records[:10]:
    print(decoder.format_record(record), end="\n\n")

title: So sánh iPhone 13 Pro Max và Samsung Galaxy Z Fold3 - Nên mua máy nào?	title : So sánh iPhone 13 Pro Max và Samsung Galaxy Z Fold3 - Nên mua máy nào ?

iPhone 13 Pro Max và Samsung Galaxy Z Fold3 là hai "chiến mã" của hai ông lớn  Apple  và  Samsung  thường được cộng đồng công nghệ đưa ra so sánh.	iPhone 13 Pro Max và Samsung Galaxy Z Fold3 là hai " chiến mã " của hai ông lớn Apple và Samsung thường được cộng đồng công nghệ đưa ra so sánh .
{"label": "COM+", "subject": ["1&&iPhone", "2&&13", "3&&Pro", "4&&Max", "6&&Samsung", "7&&Galaxy", "8&&Z", "9&&Fold3"], "object": [], "aspect": [], "predicate": ["13&&chi\u1ebfn", "14&&m\u00e3"]}

Cùng xem bài viết để xem đâu là  điện thoại  được đánh giá cao hơn nhé!	Cùng xem bài viết để xem đâu là điện thoại được đánh giá cao hơn nhé !

1.	1 .

Thiết kế	Thiết kế

Nhìn thiết kế bên ngoài ta có thể thấy được sự khác biệt rõ rệt giữa hai chiếc điện thoại.	Nhìn thiết kế bên ngoài ta có thể thấy được sự khác biệt rõ rệt giữa hai chiếc điện thoại

In [35]:
from pathlib import Path

EXPORT_DIR = "../data/export"
DEV_DIR = "../data/dev/"

sentence2record = {
    record.original: record for record in labeled_records
}

for filepath in Path(DEV_DIR).iterdir():
    with open(filepath, "r") as fd:
        content = fd.read().strip()
        raw_records = content.split("\n\n")

        with open(Path(EXPORT_DIR, filepath.name), "w") as export_fd:
            for raw_record in raw_records:
                sentence, tokenized_sentence = raw_record.split("\t")
                record = sentence2record[sentence]

                print(decoder.format_record(record), end="\n\n",file=export_fd)
